# II. Recommender Systems

## 1. Basics of Recommendation Algorithm

In [71]:
from scipy.spatial.distance import cosine
import sklearn.metrics as metrics
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation, cosine
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error

In [55]:
#TODO: load dataset into variable M
import numpy as np
import pandas as pd
NaN = np.nan

M = np.array([
    [4,    3, 2,    3],
    [1,    2, 3,    1],
    [NaN, 2, 1,    NaN],
    [4,    3, NaN, NaN]
], dtype=np.float)

print(M.shape)
print(M[0,:])
print(M[1,:])
print(M)

(4, 4)
[4. 3. 2. 3.]
[1. 2. 3. 1.]
[[ 4.  3.  2.  3.]
 [ 1.  2.  3.  1.]
 [nan  2.  1. nan]
 [ 4.  3. nan nan]]


### Compute similarities

#### Cosine

In [51]:
import math
def cosine_similarity(v1,v2, metric='cosine'):
    #metric: cosine or correlation
    if metric == 'correlation':
        v1 = v1 - np.nanmean(v1)
        v2 = v2 - np.nanmean(v2)
    "compute similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        if np.isnan(x) or np.isnan(y): continue
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

def sim_matrix(M, dimension='user', metric='cosine'):
    N = M.shape[0] if dimension == 'user' else M.shape[1]
    sim = np.zeros([N,N])
    for i in range(N):
        for j in range(N):
            if i == j:
                sim[i,j] = 0 #Cancel out the effect of self-similarity in the sums later
                continue
            if dimension == 'user':
                v1, v2 = M[i,:], M[j,:]
            else:
                v1, v2 = M[:,i], M[:,j]
            sim[i][j] = cosine_similarity(v1,v2,metric)
    return sim

In [24]:
cosine_similarity(M[0,:], M[2,:], 'cosine')

0.9922778767136677

In [52]:
sim_matrix(M, 'user')

array([[0.        , 0.79582243, 0.99227788, 1.        ],
       [0.79582243, 0.        , 0.86824314, 0.89442719],
       [0.99227788, 0.86824314, 0.        , 1.        ],
       [1.        , 0.89442719, 1.        , 0.        ]])

In [53]:
sim_matrix(M, 'item')

array([[0.        , 0.9649505 , 0.73994007, 0.99705449],
       [0.9649505 , 0.        , 0.90748521, 0.96476382],
       [0.73994007, 0.90748521, 0.        , 0.78935222],
       [0.99705449, 0.96476382, 0.78935222, 0.        ]])

#### Pearson

In [27]:
cosine_similarity(M[0,:], M[2,:], 'correlation')

0.7071067811865475

In [28]:
sim_matrix(M, 'user', 'correlation')

array([[ 0.        , -0.85280287,  0.70710678,  0.70710678],
       [-0.85280287,  0.        , -0.5547002 , -0.89442719],
       [ 0.70710678, -0.5547002 ,  0.        , -1.        ],
       [ 0.70710678, -0.89442719, -1.        ,  0.        ]])

In [65]:
sim_users = sim_matrix(M, 'user', 'cosine')

array([[0.        , 0.79582243, 0.99227788, 1.        ],
       [0.79582243, 0.        , 0.86824314, 0.89442719],
       [0.99227788, 0.86824314, 0.        , 1.        ],
       [1.        , 0.89442719, 1.        , 0.        ]])

### a) Compute the missing rating in this table using user-based collaborative filtering (CF). (Use cosine similarity, then use Pearson similarity). Assume taking all neighbors

In [60]:
def user_cf(M, metric='cosine'):
    pred = np.copy(M)
    n_users, n_items = M.shape
    avg_ratings = np.nanmean(M, axis=1)
    sim_users = sim_matrix(M, 'user', metric)
    for i in range(n_users):
        for j in range(n_items):
            if np.isnan(M[i,j]):
                pred[i,j] = sum([sim_users[user, j] for user in range(n_users)])
    return pred


In [61]:
print("User-based CF (Cosine): \n" + str(pd.DataFrame(user_cf(M, 'cosine'))))
print("User-based CF (Pearson): \n" + str(pd.DataFrame(user_cf(M, 'correlation'))))

User-based CF (Cosine): 
        0    1         2         3
0  4.0000  3.0  2.000000  3.000000
1  1.0000  2.0  3.000000  1.000000
2  2.7881  2.0  1.000000  2.894427
3  4.0000  3.0  2.860521  2.894427
User-based CF (Pearson): 
          0    1         2        3
0  4.000000  3.0  2.000000  3.00000
1  1.000000  2.0  3.000000  1.00000
2  0.561411  2.0  1.000000 -1.18732
3  4.000000  3.0 -0.847593 -1.18732


### b) Similarly, computing the missing rating using item-based CF.

In [66]:
def item_cf(M, metric='cosine'):
    pred = np.copy(M)
    n_users, n_items = M.shape
    avg_ratings = np.nanmean(M, axis=0)
    sim_items = sim_matrix(M, 'item', metric)
    for i in range(n_users):
        for j in range(n_items):
            if np.isnan(M[i,j]):
                pred[i,j] = sum([sim_items[i, item] for item in range(n_items)])
    return pred

In [67]:
print("Item-based CF (Cosine): \n" + str(pd.DataFrame(item_cf(M, 'cosine'))))
print("Item-based CF (Pearson): \n" + str(pd.DataFrame(item_cf(M, 'correlation'))))

Item-based CF (Cosine): 
          0    1         2         3
0  4.000000  3.0  2.000000  3.000000
1  1.000000  2.0  3.000000  1.000000
2  2.436778  2.0  1.000000  2.436778
3  4.000000  3.0  2.751171  2.751171
Item-based CF (Pearson): 
          0    1         2         3
0  4.000000  3.0  2.000000  3.000000
1  1.000000  2.0  3.000000  1.000000
2 -1.601534  2.0  1.000000 -1.601534
3  4.000000  3.0  1.241577  1.241577


## 2. Evaluating Recommendation Algorithms

### Predictive Accuracy

In [68]:
M_result = np.asarray([[4,3,2,3], 
                [1,2,3,1],
                [1,2,1,2],
                [4,3,2,4]])
pd.DataFrame(M_result)

,0,1,2,3
0,4,3,2,3
1,1,2,3,1
2,1,2,1,2
3,4,3,2,4


In [73]:
def evaluateRS(ratings, groundtruth, method='user_cf', metric='cosine'):
    #method: user_cf and item_cf, metric: cosine and correlation
    if method == 'user_cf':
        prediction = user_cf(ratings, metric)
    else:
        prediction = item_cf(ratings, metric)
    MSE = mean_squared_error(prediction, groundtruth)
    RMSE = round(math.sqrt(MSE),3)
    print("RMSE using {0} approach ({2}) is: {1}".format(method, RMSE, metric))
    print(pd.DataFrame(prediction))
    return

In [74]:
#TODO: evaluate the predictive accuracy
evaluateRS(M, M_result, method="user_cf", metric="cosine")
evaluateRS(M, M_result, method="user_cf", metric="correlation")
evaluateRS(M, M_result, method="item_cf", metric="cosine")
evaluateRS(M, M_result, method="item_cf", metric="correlation")

RMSE using user_cf approach (cosine) is: 0.61
        0    1         2         3
0  4.0000  3.0  2.000000  3.000000
1  1.0000  2.0  3.000000  1.000000
2  2.7881  2.0  1.000000  2.894427
3  4.0000  3.0  2.860521  2.894427
RMSE using user_cf approach (correlation) is: 1.684
          0    1         2        3
0  4.000000  3.0  2.000000  3.00000
1  1.000000  2.0  3.000000  1.00000
2  0.561411  2.0  1.000000 -1.18732
3  4.000000  3.0 -0.847593 -1.18732
RMSE using item_cf approach (cosine) is: 0.523
          0    1         2         3
0  4.000000  3.0  2.000000  3.000000
1  1.000000  2.0  3.000000  1.000000
2  2.436778  2.0  1.000000  2.436778
3  4.000000  3.0  2.751171  2.751171
RMSE using item_cf approach (correlation) is: 1.321
          0    1         2         3
0  4.000000  3.0  2.000000  3.000000
1  1.000000  2.0  3.000000  1.000000
2 -1.601534  2.0  1.000000 -1.601534
3  4.000000  3.0  1.241577  1.241577


### Ranking Accuracy

In [78]:
import scipy.stats as stats

def evaluate_rank(ratings, groundtruth, method='user_cf', metric='cosine'):
    #metric: cosine vs correlation
    if method == 'user_cf':
        prediction = user_cf(ratings, metric)
    else:
        prediction = item_cf(ratings, metric)
    
    avg_tau = 0
    n_users, n_items = M.shape
    for i in range(n_users):
        tau, p_value = stats.kendalltau(M_result[i,:], prediction[i,:])
        avg_tau += tau
    avg_tau = avg_tau / n_users
    clear_output(wait=True)
    print("Rank accuracy of " + method + " with " + metric + " metric: " + str(avg_tau))
    return avg_tau


#TODO: calculate the ranking accuracy
evaluate_rank(M, M_result, method="user_cf", metric="cosine")
evaluate_rank(M, M_result, method="user_cf", metric="correlation")
evaluate_rank(M, M_result, method="item_cf", metric="cosine")
evaluate_rank(M, M_result, method="item_cf", metric="correlation")

Ranking acc using item_cf approach (correlation) is: 0.6559016994374948


0.6559016994374948